In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import shap
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# --- This assumes your model files (moe_model.py, mop_model.py) are in the same directory ---
from moe_model import MoE as MoE_raw, MLP as MoE_Expert
from mop_model import MoPModel as MoP_raw, MoPConfig

# --- Helper class to modify the MoE Expert to output raw logits ---
class MoE_Expert_Logits(MoE_Expert):
    def __init__(self, input_size, output_size, hidden_size):
        super().__init__(input_size, output_size, hidden_size)
        self.soft = nn.Identity()

print("Block 1: Imports and setup complete.")


ModuleNotFoundError: No module named 'shap'

In [ ]:
def load_and_prepare_data(file_path):
    """
    Loads a dataset, splits it into train/val/test sets, and scales the features.
    This ensures the exact same data split is used as during training.
    """
    print(f"\n🔹 Loading data from '{file_path}'...")
    df = pd.read_csv(file_path, low_memory=False)
    X = df.drop(columns=['Dementia Status'])
    y = df['Dementia Status']
    
    # Store feature names for later use
    feature_names = X.columns.tolist()
    
    # Replicate the exact data split you used for training
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)
    
    # Scale the data in the same way
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print("Data loading, splitting, and scaling complete.")
    return X_train_scaled, X_test_scaled, feature_names

print("Block 2: Data loading function defined.")


In [ ]:
def perform_shap_analysis(model_type, model_params, model_path, background_data, test_data, feature_names):
    """
    Loads a trained PyTorch model, performs SHAP analysis, and displays the summary plot.
    """
    print(f"\n{'='*40}\n🚀 Starting SHAP Analysis for: {model_path}\n{'='*40}")
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # 1. Re-create the model with the winning architecture
    if model_type == 'MoP':
        config = MoPConfig(**model_params)
        model = MoP_raw(config)
    else: # MoE
        model = MoE_raw(**model_params)
        model.experts = nn.ModuleList([
            MoE_Expert_Logits(
                input_size=model_params['input_size'],
                output_size=2,
                hidden_size=model_params['hidden_size']
            ) for _ in range(model.num_experts)
        ])
    
    # 2. Load the saved weights
    try:
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.to(device)
        model.eval()
        print("Model and weights loaded successfully.")
    except FileNotFoundError:
        print(f"❌ ERROR: Model file not found at '{model_path}'. Please check the path.")
        return
    except Exception as e:
        print(f"❌ ERROR: Failed to load model weights. Ensure the model parameters below match the saved model.")
        print(f"   Parameters used: {model_params}")
        print(f"   Original error: {e}")
        return

    # 3. Prepare data and wrapper for SHAP
    background_tensor = torch.from_numpy(background_data).float().to(device)
    test_tensor = torch.from_numpy(test_data).float().to(device)

    # SHAP needs a function that returns raw model outputs (logits)
    def model_prediction_wrapper(x):
        if model_type == 'MoP':
            # Add sequence dimension for MoP
            pred, _, _ = model(x.unsqueeze(1))
            return pred.squeeze(1)
        else: # MoE
            pred, _ = model(x)
            return pred

    # 4. Create the SHAP explainer and calculate values
    # We use a sample of the training data as the background distribution
    explainer = shap.DeepExplainer(model_prediction_wrapper, background_tensor)
    
    print("Calculating SHAP values... (This may take a moment)")
    # We calculate SHAP values on the test set to understand predictions on unseen data
    shap_values = explainer.shap_values(test_tensor)
    
    # 5. Generate and display the summary plot
    print("\n--- SHAP Summary Plot ---")
    # For binary classification, we plot the SHAP values for the positive class (class 1)
    shap.summary_plot(shap_values[1], features=test_data, feature_names=feature_names)

print("Block 3: SHAP analysis function defined.")


In [ ]:
if __name__ == '__main__':
    # --- 1. Load both datasets ---
    X_train_full, X_test_full, feature_names_full = load_and_prepare_data('uk_biobank_dataset.csv')
    X_train_noninvasive, X_test_noninvasive, feature_names_noninvasive = load_and_prepare_data('input_data_noninvasive.csv')

    # --- 2. Define the BEST hyperparameters found for each model ---
    # ⚠️ IMPORTANT: YOU MUST UPDATE THESE DICTIONARIES WITH YOUR WINNING PARAMETERS!
    
    best_params_mop_full = {
        'input_dim': X_train_full.shape[1], 'output_dim': 2,
        'intermediate_dim': 32, 'layers': ["0,16,32", "0,8,16", "0,16,32"] # Example
    }
    
    best_params_moe_full = {
        'input_size': X_train_full.shape[1], 'output_size': 2,
        'num_experts': 6, 'hidden_size': 32, 'k': 4 # Example
    }
    
    best_params_mop_noninvasive = {
        'input_dim': X_train_noninvasive.shape[1], 'output_dim': 2,
        'intermediate_dim': 64, 'layers': ["0,8,16", "0,8,16"] # Example
    }

    best_params_moe_noninvasive = {
        'input_size': X_train_noninvasive.shape[1], 'output_size': 2,
        'num_experts': 8, 'hidden_size': 16, 'k': 3 # Example
    }

    # --- 3. Run SHAP analysis for each model ---
    
    # Analysis for MoP on the full dataset
    perform_shap_analysis('MoP', best_params_mop_full, 'best_mop_fulldataset.pth',
                          X_train_full, X_test_full, feature_names_full)
                          
    # Analysis for MoE on the full dataset
    perform_shap_analysis('MoE', best_params_moe_full, 'best_moe_fulldataset.pth',
                          X_train_full, X_test_full, feature_names_full)

    # Analysis for MoP on the non-invasive dataset
    perform_shap_analysis('MoP', best_params_mop_noninvasive, 'best_mop_noninvasivedataset.pth',
                          X_train_noninvasive, X_test_noninvasive, feature_names_noninvasive)
                          
    # Analysis for MoE on the non-invasive dataset
    # Note: You mentioned two MoE files for non-invasive, I assume one was a typo. Using 'best_moe_noninvasivedataset.pth'
    perform_shap_analysis('MoE', best_params_moe_noninvasive, 'best_moe_noninvasivedataset.pth',
                          X_train_noninvasive, X_test_noninvasive, feature_names_noninvasive)


In [2]:
pip install shap

     |████████████████████████████████| 538 kB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 66 kB 3.9 MB/s eta 0:00:011
     |████████████████████████████████| 3.7 MB 8.6 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 5.3 MB/s eta 0:00:011
     |████████████████████████████████| 43.6 MB 8.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
